Purpose: run VAE+AA for Harvard-GDP datasets

# Import libaries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['font.weight'] = 'bold'
matplotlib.rcParams['font.size'] = 18
matplotlib.rcParams['figure.titlesize'] = 18
# matplotlib.rcParams['font.style'] = 'italic'

# utility
import time
import copy as cp
from tqdm import tqdm
from collections import defaultdict
import glob
import os
import sys
from sys import stderr
import warnings
warnings.simplefilter('ignore')

# 乱数
rng = np.random.RandomState(42)
random_state = 42

# %cd your path
# %pwd

# preprocess npy (RNFLT images)


check images

In [ ]:
import matplotlib.colors as mcolors

def visualize(arr, vmin=0, vmax=1, figname=None):
    # base_cmap = plt.cm.RdYlGn
    base_cmap = plt.cm.RdYlBu_r

    colors = base_cmap(np.arange(base_cmap.N))
    colors[0] = [0, 0, 0, 1]
    new_cmap = mcolors.ListedColormap(colors)

    fig, ax = plt.subplots(figsize=(5, 5))
    im = ax.imshow(arr, cmap=new_cmap, vmin=vmin, vmax=vmax)

    ax.set_xticks([])
    ax.set_yticks([])

    plt.title("")

    plt.savefig(f"figures/samples/{figname}.pdf", transparent=True)
    plt.close()

def show_ranges(arr): return arr.min(), arr.max()

In [ ]:
import glob
import cv2

import os
os.makedirs("figures/samples", exist_ok=True)

# obtained from harvard GDP repository
files = glob.glob("datasets/RNFLT/*.npz")
files.sort()

for idx, file in enumerate(files[:10]):
    print(file)
    data = np.load(file)
    img = data['rnflt']

    vmin, vmax = show_ranges(img)
    print("min:", vmin, "max:", vmax)
    # ceiling
    vmax = 240

    visualize(img, vmin, vmax, figname=f"sample_{idx}")
    img_resized = cv2.resize(img, (26, 26), interpolation=cv2.INTER_LANCZOS4)
    visualize(img_resized, vmin, vmax, figname=f"sample_{idx}_resized")



organize RNFLT datasets

In [ ]:
import glob
import cv2

# obtained from harvard GDP repository
files = glob.glob("datasets/RNFLT/*.npz")
files.sort()
records = []

for file in files:
    ID = file.split("/")[-1].split(".")[0]
    data = np.load(file)
    if 'rnflt' not in data:
        continue
    img = data['rnflt']
    img = cv2.resize(img, (26, 26), interpolation=cv2.INTER_LANCZOS4)    
    img_flat = img.flatten()  # shape: (676,)

    row = [ID] + img_flat.tolist()
    records.append(row)

# DataFrame columns: "ID", "h1", "h2", ..., "h676"
columns = ["ID"] + [f"h{i+1}" for i in range(676)]
df_oct = pd.DataFrame(records, columns=columns)

df_oct.to_csv("datasets/data_RNFLT.csv", index=False)
print(df_oct.shape)
df_oct.head()

# preprocess CSV

ID (filename)

sex (gender): male(444), female(556)

age

race: white(743), black(162), asian(95)

hispanic: yes(25), no(944)

glaucoma: yes(443), no(557)

h1 ... h676

In [ ]:
df_info = pd.read_csv("data_harvard/data_summary.csv").iloc[:, :6]
df_info.rename(columns={"filename": "ID"}, inplace=True)
df_info["sex"] = df_info["gender"].replace("male", 1).replace("female", 0)

df_oct = pd.read_csv("data_harvard/data_RNFLT.csv")

df_all = pd.merge(df_info, df_oct, on="ID", how="inner")
df_all["ID"] = df_all["ID"].str.replace("data_", "").astype(int)
df_all.to_csv("data_harvard/data_all.csv", index=False)

print(df_all.shape)
df_all.head()